In [ ]:
import pandas as pd
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"
import yaml

from IPython.display import Markdown, display
import visualizer_helpers as vh

In [ ]:
# NOTE: SPECIALLY TAGGED PARAMETERS CELL
# Assignments in this cell may be overwritten at compile-time
# Otherwise, the below defaults to not filtering any zones
zone_set: str = 'all'
how_method: str = 'any'
affected_tazs: list = []
affected_mazs: list = []

In [ ]:
# get corresponding filter from zone_set parameter
single_filter_tazs, multi_filter_tazs = vh.get_filters(zone_set, how_method, affected_tazs)
single_filter_mazs, multi_filter_mazs = vh.get_filters(zone_set, how_method, affected_mazs)

In [ ]:
# file IO locations
with open('_quarto.yml') as f:
    config = yaml.safe_load(f)
base_dir = config['sources']['base']
build_dir = config['sources']['build']

In [ ]:
name_dict = {'escort': "Escorting", 
             'work': "Work", 
             'school': "School", 
             'othmaint': "Other-Maintenance",
             'othdiscr': "Other-Discretionary", 
             'shopping': "Shopping",
             'atwork': "At-work Subtour", 
             'eatout': "Eat Out", 
             'social' : "Social", 
             'univ': "University"}

## Coordinated Daily Activity Pattern

In [ ]:
base_persons_df = pd.read_csv(
    f"{base_dir}/final_persons.csv",
    index_col="person_id",
    usecols=[
        "person_id",
        "cdap_activity",
        'home_zone_id'
    ]
)

build_persons_df = pd.read_csv(
    f"{build_dir}/final_persons.csv",
    index_col="person_id",
    usecols=[
        "person_id",
        "cdap_activity",
        'home_zone_id'
    ]
)

In [ ]:
print(f"Filtering persons by home MAZ. Original Persons: {len(base_persons_df)}")
base_persons_df = base_persons_df[single_filter_mazs(base_persons_df.home_zone_id)]
build_persons_df = build_persons_df[single_filter_mazs(build_persons_df.home_zone_id)]
print(f"Persons after filtering: {len(base_persons_df)}")

In [ ]:
df = pd.crosstab(
    base_persons_df.cdap_activity,
    build_persons_df.cdap_activity,
    rownames=['Base'],
    colnames=['Build'],
    margins=True,
    margins_name='Total'
)

df

## Mandatory Tour Frequency

In [ ]:
# mandatory tour frequency is a decision made by persons, we should apply the filter on person

base_persons_df = pd.read_csv(
    f"{base_dir}/final_persons.csv",
    index_col="person_id",
    usecols=[
        "person_id",
        "mandatory_tour_frequency",
        "home_zone_id",
        "ptype"
    ]
)
# keep workers and students
base_persons_df = base_persons_df[base_persons_df.ptype.isin([1,2,3,6,7,8])]
base_persons_df.loc[base_persons_df.ptype.isin([1,2]), 'person_type'] = "workers"
base_persons_df.loc[base_persons_df.ptype.isin([3,6,7,8]), 'person_type'] = "students"

build_persons_df = pd.read_csv(
    f"{build_dir}/final_persons.csv",
    index_col="person_id",
    usecols=[
        "person_id",
        "mandatory_tour_frequency",
        "home_zone_id",
        "ptype"
    ]
)
build_persons_df = build_persons_df[build_persons_df.ptype.isin([1,2,3,6,7,8])]
build_persons_df.loc[build_persons_df.ptype.isin([1,2]), 'person_type'] = "workers"
build_persons_df.loc[build_persons_df.ptype.isin([3,6,7,8]), 'person_type'] = "students"

In [ ]:
print(f"Filtering persons by home MAZ. Original Persons: {len(base_persons_df)}")
base_persons_df = base_persons_df[single_filter_mazs(base_persons_df.home_zone_id)]
build_persons_df = build_persons_df[single_filter_mazs(build_persons_df.home_zone_id)]
print(f"Persons after filtering: {len(base_persons_df)}")

In [ ]:
# | output: asis
# above comment is needed for Quarto to render subtabs correctly

for person_type in ['workers', 'students']:
    df = pd.crosstab(
        base_persons_df[base_persons_df.person_type == person_type].mandatory_tour_frequency.fillna('None'),
        build_persons_df[build_persons_df.person_type == person_type].mandatory_tour_frequency.fillna('None'),
        rownames=['Base'],
        colnames=['Build'],
        margins=True,
        margins_name='Total',
        dropna=False
    )
    
    display(Markdown(f"### {person_type.capitalize()} Mandatory Tour Frequency"))
    display(df)

## Non-Mandatory Tour Frequency

In [ ]:
# Tour Frequency is a person choice, so we should apply the filter on person

base_persons_df = pd.read_csv(
    f"{base_dir}/final_persons.csv", 
    index_col='person_id',
    usecols=['person_id', 'home_zone_id'],
)

build_persons_df = pd.read_csv(
    f"{build_dir}/final_persons.csv",
    index_col='person_id',
    usecols=['person_id', 'home_zone_id'],
)

In [ ]:
print(f"Filtering persons by home MAZ. Original Persons: {len(base_persons_df)}")
base_persons_df = base_persons_df[single_filter_mazs(base_persons_df.home_zone_id)]
build_persons_df = build_persons_df[single_filter_mazs(build_persons_df.home_zone_id)]
print(f"Persons after filtering: {len(base_persons_df)}")

In [ ]:
# read in unfiltered tours for base and build

base_tours_df = pd.read_csv(f"{base_dir}/final_tours.csv", 
                        index_col='tour_id', 
                        usecols=['tour_id','tour_category','primary_purpose','person_id'])

build_tours_df = pd.read_csv(f"{build_dir}/final_tours.csv", 
                         index_col='tour_id', 
                         usecols=['tour_id','tour_category','primary_purpose','person_id'])

# keep non-mandatory tours only
base_tours_df = base_tours_df[base_tours_df.tour_category == 'non_mandatory']
build_tours_df = build_tours_df[build_tours_df.tour_category == 'non_mandatory']

In [ ]:
# count non-mandatory tours by person
base_tour_counts_df = base_tours_df.groupby('person_id').size()
build_tour_counts_df = build_tours_df.groupby('person_id').size()
base_tour_counts_df = base_tour_counts_df.rename('non-mandatory')
build_tour_counts_df = build_tour_counts_df.rename('non-mandatory')

# reindex to ensure all persons are included, filling missing with 0
base_person_tour_counts_df = base_tour_counts_df.reindex(base_persons_df.index, fill_value=0)
build_person_tour_counts_df = build_tour_counts_df.reindex(base_persons_df.index, fill_value=0)

display(Markdown(f"### Non-Mandatory Tour Counts\n"))
df = pd.crosstab(
    base_person_tour_counts_df, 
    build_person_tour_counts_df, 
    rownames=['Base'], 
    colnames=['Build'], 
    margins=True, 
    margins_name='Total'
)

df

In [ ]:
# | output: asis
# above comment is needed for Quarto to render subtabs correctly

for key, value in name_dict.items():
    
    if len(base_tours_df[base_tours_df.primary_purpose == key]) == 0:
        continue

    # count tours by primary purpose
    base_tour_counts_df = base_tours_df[base_tours_df.primary_purpose == key].groupby('person_id').size()
    build_tour_counts_df = build_tours_df[build_tours_df.primary_purpose == key].groupby('person_id').size()
    base_tour_counts_df = base_tour_counts_df.rename(value)
    build_tour_counts_df = build_tour_counts_df.rename(value)

    # reindex to ensure all persons are included, filling missing with 0
    base_person_tour_counts_df = base_tour_counts_df.reindex(base_persons_df.index, fill_value=0)
    build_person_tour_counts_df = build_tour_counts_df.reindex(base_persons_df.index, fill_value=0)

    df = pd.crosstab(
        base_person_tour_counts_df, 
        build_person_tour_counts_df, 
        rownames=['Base'], 
        colnames=['Build'], 
        margins=True, 
        margins_name='Total'
    )

    display(Markdown(f"\n### {value} Tour Counts"))
    display(df)